In [56]:
import cv2
print("Current version of openCV=", cv2.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [57]:
left_calibration_path = './calibration/fisheye/{}.png'

In [58]:
CHECKERBOARD = (7,7)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+ \
cv2.fisheye.CALIB_CHECK_COND+\
cv2.fisheye.CALIB_FIX_SKEW

cv2.fisheye
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./calibration/fisheye/*.png')
for fname in images:
    print(fname)
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
DIM = _img_shape[::-1]
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

./calibration/fisheye/3.png
./calibration/fisheye/21.png
./calibration/fisheye/19.png
./calibration/fisheye/12.png
./calibration/fisheye/0.png
./calibration/fisheye/20.png
./calibration/fisheye/13.png
./calibration/fisheye/14.png
./calibration/fisheye/16.png
./calibration/fisheye/10.png
./calibration/fisheye/27.png
./calibration/fisheye/23.png
./calibration/fisheye/28.png
./calibration/fisheye/15.png
./calibration/fisheye/1.png
./calibration/fisheye/2.png
./calibration/fisheye/5.png
./calibration/fisheye/7.png
./calibration/fisheye/26.png
./calibration/fisheye/25.png
./calibration/fisheye/4.png
./calibration/fisheye/11.png
./calibration/fisheye/6.png
./calibration/fisheye/24.png
./calibration/fisheye/8.png
./calibration/fisheye/18.png
./calibration/fisheye/17.png
./calibration/fisheye/9.png
./calibration/fisheye/29.png
./calibration/fisheye/22.png


error: OpenCV(4.1.1) /io/opencv/modules/calib3d/src/fisheye.cpp:1421: error: (-3:Internal error) CALIB_CHECK_COND - Ill-conditioned matrix for input array 3 in function 'CalibrateExtrinsics'


In [ ]:
img_path = "./calibration/fisheye/{}.png"
dst = "./calibration/fisheye_calibrated/{}.png"
for img_id in range(len(images)):
    img = cv2.imread(img_path.format(img_id))
    h,w = img.shape[:2]
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    cv2.imshow("undistorted", undistorted_img)
    cv.imwrite(dst.format(img_id), undistorted_img)
    cv2.waitKey(500)
#cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()
